# 5. Influence factors: Berry Chemistry

In [1]:
#!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/berry-chemistry

In [2]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/berry-chemistry'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/berry-chemistry


In [3]:
%env TMPDIR=/scratch/lfloerl/tmpdata

env: TMPDIR=/scratch/lfloerl/tmpdata


In [11]:
from qiime2 import Visualization
import qiime2 as q2
from qiime2 import Visualization
from qiime2 import Artifact
from qiime2 import Metadata
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.feature_table.methods import (filter_samples, filter_seqs)

from qiime2.plugins import diversity as q2d

%matplotlib inline

## Analyse HPLC data in the context of veraison and harvest seperately

### Diversity of subsets

In [ ]:
%%bash 

# Define the time points
time_points=('Veraison' 'Harvest')

# Loop through each time point
for time_point in "${time_points[@]}"
do 
    echo "Processing $time_point"
    
    # Create directory for the time point and navigate into it
    mkdir -p "./$time_point"
    pushd "./$time_point" > /dev/null

    # Filter table and rep seqs based on Time_point and where they have HPLC data 
    qiime feature-table filter-samples \
      --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_filtered_table.qza \
      --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_BerryChemistry.tsv \
      --p-where "[Time_point]='$time_point' AND [Tartrate_gL] IS NOT NULL" \
      --o-filtered-table must_${time_point}_filtered_table.qza

    qiime feature-table filter-seqs \
        --i-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
        --i-table must_${time_point}_filtered_table.qza \
        --o-filtered-data must_${time_point}_filtered_rep_seqs.qza

    # Core diversity metrics
    qiime diversity core-metrics \
        --i-table must_${time_point}_filtered_table.qza \
        --p-sampling-depth 5000 \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_BerryChemistry.tsv \
        --p-n-jobs 5 \
        --output-dir cm5000/

    # Kmer diversity
    qiime kmerizer core-metrics \
        --i-sequences must_${time_point}_filtered_rep_seqs.qza \
        --i-table must_${time_point}_filtered_table.qza \
        --p-sampling-depth 5000 \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_BerryChemistry.tsv \
        --p-n-jobs auto \
        --p-max-features 5000 \
        --output-dir kmer_cm5000/

    # Return to the parent directory
    popd > /dev/null
done


### PERMANOVA thereof

In [69]:
%%bash

# Formula for PERMANOVA with berry chemistry variables
formula='Plot_ID + Year + Glucose_gL + Tartrate_gL + Malate_gL + Fructose_gL'

beta_metrics=('bray_curtis' 'jaccard')
time_points=('Veraison' 'Harvest')

# Loop through each time point
for time_point in "${time_points[@]}"
do 
    echo "Processing PERMANOVA for $time_point"
    
    div_dir="./$time_point/cm5000"
    # PERMANOVA with core metrics
    for m in "${beta_metrics[@]}"
    do
        qiime diversity adonis \
            --i-distance-matrix "${div_dir}/${m}_distance_matrix.qza" \
            --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_BerryChemistry.tsv \
            --p-formula "$formula" \
            --p-n-jobs 5 \
            --o-visualization "${div_dir}/adonis_${m}.qzv"
    done

    kmer_dir="./$time_point/kmer_cm5000"
    # PERMANOVA with kmer core metrics
    for m in "${beta_metrics[@]}"
    do
        qiime diversity adonis \
            --i-distance-matrix "${kmer_dir}/${m}_distance_matrix.qza" \
            --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_BerryChemistry.tsv \
            --p-formula "$formula" \
            --p-n-jobs 5 \
            --o-visualization "${kmer_dir}/adonis_${m}.qzv"
    done
done


Processing PERMANOVA for Veraison
Saved Visualization to: ./Veraison/cm5000/adonis_bray_curtis.qzv
Saved Visualization to: ./Veraison/cm5000/adonis_jaccard.qzv
Saved Visualization to: ./Veraison/kmer_cm5000/adonis_bray_curtis.qzv
Saved Visualization to: ./Veraison/kmer_cm5000/adonis_jaccard.qzv
Processing PERMANOVA for Harvest
Saved Visualization to: ./Harvest/cm5000/adonis_bray_curtis.qzv
Saved Visualization to: ./Harvest/cm5000/adonis_jaccard.qzv
Saved Visualization to: ./Harvest/kmer_cm5000/adonis_bray_curtis.qzv
Saved Visualization to: ./Harvest/kmer_cm5000/adonis_jaccard.qzv


### Check it out! 

#### Harvest

In [70]:
Visualization.load('Harvest/cm5000/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: c19ea7a7-ee0b-4285-90aa-10479b1efec8>

In [71]:
Visualization.load('Harvest/cm5000/adonis_jaccard.qzv')

<visualization: Visualization uuid: 50eeb9d6-f293-4df6-af77-8ad595896f0f>

In [72]:
Visualization.load('Harvest/kmer_cm5000/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: 04742e8a-ce5c-4e38-8582-c4aca95420c0>

In [73]:
Visualization.load('Harvest/kmer_cm5000/adonis_jaccard.qzv')

<visualization: Visualization uuid: 9c2a8e4c-aee7-477b-94e7-c078aca7ac89>

#### Veraison

In [74]:
Visualization.load('Veraison/cm5000/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: ba23ad9a-0978-4d08-a680-e66e1ea31da7>

In [75]:
Visualization.load('Veraison/cm5000/adonis_jaccard.qzv')

<visualization: Visualization uuid: 9facb96a-d38c-434d-bae7-d1825c143c9d>

In [76]:
Visualization.load('Veraison/kmer_cm5000/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: dc7e03f9-7b81-4195-aaf3-d771510553c5>

In [77]:
Visualization.load('Veraison/kmer_cm5000/adonis_jaccard.qzv')

<visualization: Visualization uuid: 9f2e33e8-f0db-4956-b3ab-876e01b4fecf>